In [1]:
#import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import os
import sys
import matplotlib.pyplot as plt

In [2]:
deeplabv3plus_dir="./src"
sys.path.append(deeplabv3plus_dir)

In [3]:
gpu_options = tf.compat.v1.GPUOptions(visible_device_list="2", allow_growth=True)
config = tf.compat.v1.ConfigProto(gpu_options = gpu_options)
tf.compat.v1.enable_eager_execution(config=config)

In [4]:
from model import deeplab_v3plus_transfer_resnet_os16
#from image_utils import make_x_from_image_paths,make_y_from_image_paths,convert_y_to_image_array
from data_gen import DataGenerator

In [5]:
import importlib
importlib.reload(sys.modules['model'])

<module 'model' from './src/model.py'>

In [6]:
out_dir = "resnet50_transfer_os16"
os.makedirs(out_dir, exist_ok=True)

In [7]:
n_categories = 21
image_size = (512,512)

In [8]:
seg_img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/SegmentationClass"
img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/JPEGImages"
train_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt"
valid_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt"

with open(train_set_path) as f:
    train_img_names = f.read().split("\n")[:-1]
with open(valid_set_path) as f:
    valid_img_names = f.read().split("\n")[:-1]

img_paths = [os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))]
seg_img_paths = [os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))]

valid_x_paths = [os.path.join(img_dir,valid_img_names[i]) + ".jpg" for i in range(len(valid_img_names))]
valid_y_paths = [os.path.join(seg_img_dir,valid_img_names[i]) + ".png" for i in range(len(valid_img_names))]

In [9]:
batch_size=16
n_epochs=300

In [10]:
encoder = keras.applications.ResNet50(input_shape=(512,512,3), weights="imagenet", include_top=False)
preprocess = keras.applications.resnet50.preprocess_input
layer_name_to_decoder = "conv2_block3_out"
encoder_end_layer_name = "conv4_block6_out"
model = deeplab_v3plus_transfer_resnet_os16(n_categories, encoder, layer_name_to_decoder, encoder_end_layer_name)

In [11]:
model.summary()

Model: "resnet50_deeplab-v3plus"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________

In [12]:
train_data_gen = DataGenerator(n_categories, image_size, batch_size, img_paths    , seg_img_paths, preprocess=preprocess, augmentation=True )
valid_data_gen = DataGenerator(n_categories, image_size, batch_size, valid_x_paths, valid_y_paths, preprocess=preprocess, augmentation=False)

In [13]:
loss_function = tf.keras.losses.categorical_crossentropy
opt = tf.keras.optimizers.Adam()
iou =  keras.metrics.MeanIoU(num_classes=21)
model.compile(optimizer=opt, loss=loss_function, metrics=["accuracy", iou])

In [14]:
filepath = os.path.join(out_dir,'{epoch:06d}.h5')
cp_cb = keras.callbacks.ModelCheckpoint(filepath, 
                                        monitor='val_loss', 
                                        verbose=0, 
                                        save_best_only=True, 
                                        save_weights_only=False, 
                                        mode='auto')

In [ ]:
#hist = model.fit_generator(data_gen, validation_data=(valid_x, valid_y), epochs=par.n_epochs, steps_per_epoch=par.n_batch, callbacks=[cp_cb])
hist = model.fit_generator(train_data_gen,
                           epochs=n_epochs,
                           steps_per_epoch=len(train_data_gen),
                           validation_data=valid_data_gen,
                           validation_steps=len(valid_data_gen),
                           shuffle = True,
                           workers=8,
                           use_multiprocessing=True,
                           callbacks=[cp_cb])
#hist = model.fit_generator(data_gen, epochs=par.n_epochs, steps_per_epoch=par.n_batch, workers=8, use_multiprocessing=True)

Epoch 1/300


W1129 10:26:27.069119 140736002870480 deprecation.py:323] From /home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


92/92 [==============================] - 303s 3s/step - loss: 2.4584 - acc: 0.5804 - mean_io_u: 0.4762 - val_loss: 2.5418 - val_acc: 0.7476 - val_mean_io_u: 0.4762
Epoch 2/300
92/92 [==============================] - 285s 3s/step - loss: 2.1213 - acc: 0.6919 - mean_io_u: 0.4762 - val_loss: 2.2457 - val_acc: 0.7476 - val_mean_io_u: 0.4762
Epoch 3/300
92/92 [==============================] - 278s 3s/step - loss: 1.9482 - acc: 0.7218 - mean_io_u: 0.4762 - val_loss: 1.9772 - val_acc: 0.7487 - val_mean_io_u: 0.4762
Epoch 4/300
92/92 [==============================] - 289s 3s/step - loss: 1.7805 - acc: 0.7488 - mean_io_u: 0.4762 - val_loss: 1.7287 - val_acc: 0.7725 - val_mean_io_u: 0.4762
Epoch 5/300
92/92 [==============================] - 278s 3s/step - loss: 1.6341 - acc: 0.7643 - mean_io_u: 0.4762 - val_loss: 1.4438 - val_acc: 0.8288 - val_mean_io_u: 0.4762
Epoch 6/300
92/92 [==============================] - 283s 3s/step - loss: 1.5153 - acc: 0.7834 - mean_io_u: 0.4762 - val_loss: 1.262

In [ ]:
plt.figure(figsize=(30,10))

plt.subplot(1,3,1)
plt.plot(hist.history["loss"], label="loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.legend()

plt.subplot(1,3,2)
plt.plot(hist.history["acc"], label="acc")
plt.plot(hist.history["val_acc"], label="val_acc")
plt.legend()
plt.savefig(os.path.join(out_dir,'losscurve.png'))

In [ ]:
model.save(os.path.join(out_dir,'final_epoch.h5'))

In [ ]:
import numpy as np
for key in sorted(hist.history.keys()):
    np.savetxt(os.path.join(out_dir,key+'.txt'),np.array(hist.history[key]))